# Problems:
* Decimal is unsupported -> conversion to float
* No constraints besides the unique constraints -> all checks should be performed on backend

In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

import psycopg2

from openpyxl import Workbook
from openpyxl.styles import Alignment

import tabulate

from time import time
from typing import List, Tuple, Optional

In [2]:
NEO_LOGIN = "neo4j"
NEO_PASSWORD = "bitnami"

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO_LOGIN, NEO_PASSWORD))

In [18]:
PG_LOGIN = "DMD2user"
PG_PASS = "DMD2pgPass"
PG_DB = "dvdrental"

conn = psycopg2.connect(database=PG_DB, user=PG_LOGIN,
                       password=PG_PASS, host="localhost", port="5432")

cursor = conn.cursor()

In [24]:
steps = 42
current_step = 1

def report():
    global steps, current_step
    print(f"{current_step}/{steps} done")
    current_step += 1


def proper_type(column_name:str, column_type:str) -> str:
    if column_type in ["integer", "int", "smallint", "bigint"]:
        return f"toInteger(row.{column_name})"
    elif column_type in ["numeric", "float", "decimal"]:
        return f"toFloat(row.{column_name})"
    elif column_type in ["boolean"]:
        return f"toBoolean(row.{column_name})"
    elif column_type in ["date"]:
        # return f'date(datetime({{epochmillis:apoc.date.parse("row.{column_name}", "s", "yyyy-MM-dd")}}))'
        return f'apoc.date.parse(row.{column_name}, \'s\', "yyyy-MM-dd")'
    elif column_type.startswith("timestamp"):
        # return f"datetime({{epochmillis:toInteger(apoc.date.parse(row.{column_name}))}})"
        return f"apoc.date.parse(row.{column_name})"
    else:
        return f"row.{column_name}"

def print_unique_constraint(table:str) ->str:
    return f"CREATE CONSTRAINT ON ({table}:{table.capitalize()}) ASSERT {table}.{table}_id IS UNIQUE"

def print_transfer(table_name:str, with_index=True) -> Tuple[str, str]:
    export = f"copy {table_name} to '/tmp/{table_name}.csv' DELIMITER ',' CSV HEADER;"
    cursor.execute(export)
    import_statement = "USING PERIODIC COMMIT\n" \
                       f"LOAD CSV WITH HEADERS FROM 'file:/{table_name}.csv' AS row\n" \
                       f"MERGE ({table_name.lower()}: {table_name.capitalize()} {{{table_name.lower()}_id:toInteger(row.{table_name.lower()}_id)}})\n" \
                       "\tON CREATE SET\n"

    cursor.execute("select column_name, data_type from INFORMATION_SCHEMA.COLUMNS where table_name = %s ;", (table_name,))
    
    props = []
    for column in cursor.fetchall():
        column_name, column_type = column
        if column_name.endswith("_id"):
            continue
        props.append(f"{table_name}.{column_name} = {proper_type(column_name, column_type)}")
    
    return print_unique_constraint(table_name), import_statement + "\t\t{data};".format(data=",\n\t\t".join(props))

def many2many(table_name:str, from_table:str, to_table:str, rel_name:str) -> str:
    export = f"copy {table_name} to '/tmp/{table_name}.csv' DELIMITER ',' CSV HEADER;"
    cursor.execute(export)
    cursor.execute("select column_name, data_type from INFORMATION_SCHEMA.COLUMNS where table_name = %s ;", (table_name,))
    
    props = []
    for column in cursor.fetchall():
        column_name, column_type = column
        if column_name.endswith("_id"):
            continue
        props.append(f"{column_name}: {proper_type(column_name, column_type)}")
        
    import_statement = "USING PERIODIC COMMIT\n" \
                       f"LOAD CSV WITH HEADERS FROM 'file:/{table_name}.csv' AS row\n" \
                       f"MATCH ( {from_table} :{from_table.capitalize()} {{ {from_table}_id: {proper_type(f'{from_table}_id', 'int')} }})\n" \
                       f"MATCH ( {to_table}   :{to_table.capitalize()} {{ {to_table}_id: {proper_type(f'{to_table}_id', 'int')} }})\n" \
                       f"MERGE ( {from_table}) - [:{rel_name.upper()}{{ {','.join(props)} }}] -> ({to_table});"
    return import_statement
            
def transfer_relation(table_name:str, to_table:str, rel_name:str, 
                      last_update_to_rel: bool = True, fk:Optional[str] = None) -> str:
    if fk is None:
        fk = f"{to_table}_id"
    export = f"copy (select {table_name}_id, {fk} as {to_table}_id " \
             f"{', last_update' if last_update_to_rel else ''}" \
             f" from {table_name}) to '/tmp/{table_name}_{to_table}_rel.csv' DELIMITER ',' CSV HEADER;"
    cursor.execute(export)
    
    props = []
    if last_update_to_rel:
        props.append(f"last_update: {proper_type('last_update', 'timestamp')}")
    
    import_statement = "USING PERIODIC COMMIT\n" \
                       f"LOAD CSV WITH HEADERS FROM 'file:/{table_name}_{to_table}_rel.csv' AS row\n" \
                       f"MATCH ( {table_name} :{table_name.capitalize()} {{ {table_name}_id: {proper_type(f'{table_name}_id', 'int')} }})\n" \
                       f"MATCH ( {to_table}   :{to_table.capitalize()} {{ {to_table}_id: {proper_type(f'{to_table}_id', 'int')} }})\n" \
                       f"MERGE ( {table_name}) - [:{rel_name.upper()}{{ {','.join(props)} }}] -> ({to_table});"
    return import_statement


def execute_statements(statements: List[str]) -> None:
    with driver.session() as session:
        for statement in statements:
            if statement is None:
                continue
            try:
                session.run(statement)
                report()
            except ClientError as e:
                print(e.message)


start = time()

import_statements = []

tables = ["category", "film", "language", "actor", "staff",
          "payment", "rental", "inventory",
          "customer", "address", "city", "country", "store"]
# with driver.session() as session:
for table in tables:
    #     index, table_creation = print_transfer(table)
    #     session.run(index)
    #     session.run(table_creation)
    import_statements.extend(print_transfer(table))
    
import_statements.extend([
    many2many("film_category", "film", "category", "IN_CATEGORY"),
    many2many("film_actor", "actor", "film", "FILMED_IN"),
    transfer_relation("film", "language", "IN_LANGUAGE"),
    transfer_relation("inventory", "film", "RENTS_FILM"),
    transfer_relation("rental", "inventory", "RENTS"),
    transfer_relation("rental", "customer", "RENTED_TO"),
    transfer_relation("rental", "staff", "RENTED_BY"),
    transfer_relation("payment", "rental", "PAID_FOR", last_update_to_rel=False),
    transfer_relation("payment", "customer", "PAID_BY", last_update_to_rel=False),
    transfer_relation("payment", "staff", "ACCEPTED_BY", last_update_to_rel=False),
    transfer_relation("customer", "address", "LIVES_AT"),
    transfer_relation("staff", "address", "LIVES_AT"),
    transfer_relation("store", "address", "LOCATED_AT"),
    transfer_relation("address", "city", "SITUATED_IN"),
    transfer_relation("city", "country", "SITUATED_IN"),
    transfer_relation("store", "staff", "MANAGED_BY", fk="manager_staff_id"),
])

execute_statements(import_statements)

print(f"Time elapsed: {time()-start} seconds")

1/42 done
2/42 done
3/42 done
4/42 done
5/42 done
6/42 done
7/42 done
8/42 done
9/42 done
10/42 done
11/42 done
12/42 done
13/42 done
14/42 done
15/42 done
16/42 done
17/42 done
18/42 done
19/42 done
20/42 done
21/42 done
22/42 done
23/42 done
24/42 done
25/42 done
26/42 done
27/42 done
28/42 done
29/42 done
30/42 done
31/42 done
32/42 done
33/42 done
34/42 done
35/42 done
36/42 done
37/42 done
38/42 done
39/42 done
40/42 done
41/42 done
42/42 done
Time elapsed: 13.10915756225586 seconds


# Queries

In [15]:
with driver.session() as session:
    data = session.run (
        "MATCH (r:Rental)\n"
        "WITH r\n"
        "ORDER BY r.rental_date DESC LIMIT 1\n"
        "with datetime({epochmillis:r.rental_date}).year as most_recent_year\n"
        
        "match (cat2:Category)<-[:IN_CATEGORY]-(f2:Film)<-[:RENTS_FILM]-(:Inventory)<-[:RENTS]-(r2:Rental)-[:RENTED_TO]->\n"
        "(customer:Customer)<-[:RENTED_TO]-(r1:Rental)-[:RENTS]->(:Inventory)-[:RENTS_FILM]->(f1:Film)-[:IN_CATEGORY]->(cat1:Category)\n"
        
        "where cat2 <> cat1 and f1 <> f2 and\n"
        "datetime({epochmillis:r1.rental_date}).year = most_recent_year and datetime({epochmillis:r2.rental_date}).year = most_recent_year\n"
        "return DISTINCT customer.customer_id as customer_id, customer.first_name as first_name, customer.last_name as last_name"
    )
    q1 = [f'{x["customer_id"]}: {x["first_name"]} {x["last_name"]}' for x in data]
    print("Query1:\nPeople, who rented movies of at least two different categories during the most recent year\n\t","\n\t".join(q1))
    

Query1:
People, who rented movies of at least two different categories during the most recent year
	 15: Helen Harris
	42: Carolyn Perez
	43: Christine Roberts
	53: Heather Morris
	75: Tammy Sanders
	107: Florence Woods
	155: Gail Knight
	163: Cathy Spencer
	175: Annette Olson
	208: Lucy Wheeler
	216: Natalie Meyer
	267: Margie Wade
	269: Cassandra Walters
	284: Sonia Gregory
	354: Justin Ngo
	361: Lawrence Lawton
	448: Miguel Betancourt
	457: Bill Gavin
	516: Elmer Noe
	560: Jordan Archuleta
	576: Morris Mccarter


In [ ]:
 with driver.session() as session:
    data = session.run(
        "match (a1:Actor) - [:FILMED_IN] -> (f:Film) <- [:FILMED_IN] - (a2:Actor)\n"
        "where a1.actor_id < a2.actor_id\n"
        "return a1.actor_id as actor1_id, a2.actor_id as actor2_id, count(*) as count, "
        "a1.first_name as a1_first_name, a1.last_name as a1_last_name, a2.first_name as a2_first_name, a2.last_name as a2_last_name\n"
        "order by actor1_id, actor2_id"
    )
    q2 = [(x["actor1_id"], x["actor2_id"], x["count"], 
           f"{x['a1_first_name']} {x['a1_last_name']}", f"{x['a2_first_name']} {x['a2_last_name']}") for x in data]

    wb = Workbook()
    ws = wb.active
    m = -1
    for t in q2:
        actor1, actor2, count, name1, name2 = t
        ws.cell(row=actor1 + 1 + 1, column=actor2 + 1 + 1, value=count)
        ws.cell(row=actor2 + 1 + 1, column=actor1 + 1 + 1, value=count)
        
        ws.cell(row=actor1 + 1 + 1, column=1, value=name1)
        ws.cell(row=actor2 + 1 + 1, column=1, value=name2)
        
        ws.cell(row=1, column=actor1 + 1 + 1, value=name1).alignment = Alignment(text_rotation=85)
        ws.cell(row=1, column=actor2 + 1 + 1, value=name2).alignment = Alignment(text_rotation=85)
        
        m = max(m, actor1, actor2)
    
    for i in range(2, ws.max_row + 1):
        if i - 1 <= m:
            ws.cell(row=2, column=i + 1, value=i - 1)
            ws.cell(row=i + 1, column=2, value=i - 1)
        for j in range(3, ws.max_column + 1 ):
            cell = ws.cell(row=i, column=j)
            if cell.value is None:
                cell.value = 0
                
    wb.save("Query2.xlsx")
    print('See the result of query2 in the file "Query2.xlsx"')

In [14]:
# Query 3
def get_customer_history(customer_id:int) -> Tuple[Tuple[int,str,int,str, int]]:    
    with driver.session() as session:
        data = session.run (
            "match (c:Customer{customer_id:$cust_id})<-[:RENTED_TO]-(:Rental)-[:RENTS]->(:Inventory)-[:RENTS_FILM]->\n"
            "(f:Film)-[:IN_CATEGORY]->(cat:Category)\n"
            "return f.film_id as film_id, f.title as title,\n"
            "cat.category_id as category_id, cat.name as category, count(*) as count\n"
            "order by count desc", cust_id = customer_id
        )
        
        return tuple( (x["film_id"], x["title"], x["category_id"], x["category"], x["count"]) for x in data)

data = get_customer_history(12)
print("Query3:")
print(tabulate.tabulate(data, ["film_id", "title", "category_id", "category", "rental_count"]))

  film_id  title                    category_id  category       rental_count
---------  ---------------------  -------------  -----------  --------------
      737  Rock Instinct                     11  Horror                    2
      301  Family Sweet                      11  Horror                    1
      772  Sea Virgin                         7  Drama                     1
     1000  Zorro Ark                          5  Comedy                    1
      818  Something Duck                     7  Drama                     1
      978  Wisdom Worker                      5  Comedy                    1
      922  Undefeated Dalmations             11  Horror                    1
      249  Dracula Crystal                    4  Classics                  1
      924  Unforgiven Zoolander              14  Sci-Fi                    1
      949  Volcano Texas                     10  Games                     1
      186  Craft Outfield                     7  Drama                     1

In [19]:
# Query 4
def get_customer_recommendations(customer_id:int, limit : Optional[int]= None):
    with driver.session() as session:
        data = session.run(
            "match (:Customer{customer_id:$cust_id}) <- [:RENTED_TO] - (:Rental) - [:RENTS] -> (:Inventory) - [views:RENTS_FILM] -> (watched:Film)\n"
            "<-[:FILMED_IN] - (a:Actor)- [conn:FILMED_IN] -> (rec:Film), (watched) - [:IN_CATEGORY] -> (cat:Category) <- [:IN_CATEGORY] - (rec)\n"
            "where rec <> watched\n"
            "return rec.film_id as recommendation_film_id, rec.title as recommendation_film_title, cat.name as category,\n"
            "count(distinct conn) + count(distinct views) as recommendation_rate\n"
            "order by recommendation_rate desc\n"
            f"{'limit $lim' if limit else ''}",
            cust_id=customer_id, lim=limit
        )
        
        return tuple((x["recommendation_film_id"], x["recommendation_film_title"], x["category"], x["recommendation_rate"]) for x in data)
print("Query4:")
data = get_customer_recommendations(12, limit=5)
print("Recommendations:\n", tabulate.tabulate(data, ["film_id", "title", "category", "recommendation_rate"]))

Query4:
Recommendations:
   film_id  title               category      recommendation_rate
---------  ------------------  ----------  ---------------------
      990  World Leathernecks  Horror                          8
       34  Arabia Dogma        Horror                          7
      146  Chitty Lock         Drama                           6
      666  Paycheck Wait       Drama                           5
       65  Behavior Runaway    Horror                          5


In [ ]:
def get_separation_degrees(actor_id:int):
    with driver.session() as session:
        data = session.run(
            "match p=shortestPath(\n"
            "	(actor:Actor{actor_id:$a_id}) -[r:FILMED_IN*..50] -(next:Actor)\n"
            ")\n"
            "where actor<>next\n"
            "return next.actor_id as to_id, next.first_name as to_name, next.last_name as to_lastname, size(r)/2 as hops\n"
            "order by to_id",
            a_id = actor_id
        )
        return tuple( (x["to_id"], f'{x["to_name"]} {x["to_lastname"]}', x["hops"]) for x in data)

print("Query5:")
print(tabulate.tabulate(get_separation_degrees(12), ["to (id)", "name", "degree"]))

In [ ]:
driver.close()
conn.close()